In [1]:
from difflib import SequenceMatcher
import re
import os
from tqdm import tqdm
from wordsegment import load, segment
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import string 

load()

def wsp_tokenizer(text):
    return text.split(" ")

In [2]:


            for tag, i1, i2, j1, j2 in sm.get_opcodes():
                        print ("%7s a[%d:%d] (%s) b[%d:%d] (%s)" %
                            (tag, i1, i2, 'a', j1, j2, 'a'))
                        if tag == 'delete' or tag == 'equal' or tag == 'replace':
                            print('\n', text[i1:i2], '\n-----------------\n', preprocessed_text[j1:j2], '\n')

NameError: name 'sm' is not defined

In [2]:
class Preprocessor:
    def __init__(self, preprocessing_options):
        self.po = preprocessing_options
        if preprocessing_options.get('hashtag') or preprocessing_options.get('char_normalization'):
            annotate = set()

            if preprocessing_options.get('hashtag'):
                if preprocessing_options.get('hashtag').get('replace'):
                    annotate.add('hashtag')
            if preprocessing_options.get('char_normalization'):
                annotate.add('elongated')

            annotate.add('censored')

            self.text_preprocessor = TextPreProcessor(annotate=annotate,
                # corpus from which the word statistics are going to be used for word segmentation 
                segmenter="twitter", 

                # corpus from which the word statistics are going to be used for spell correction
                corrector="twitter", 
                unpack_hashtags=True,  # perform word segmentation on hashtags
                unpack_contractions=False,  # Unpack contractions (can't -> can not)
                spell_correct_elong=False,  # spell correction for elongated words

                # select a tokenizer. You can use SocialTokenizer, or pass your own tokenizer, should take as input a string and return a list of tokens
                tokenizer=wsp_tokenizer
            )
            
    def preprocess_and_sync(self, input_file_path, output_file_path, input_ann_file_path, output_ann_file_path):
        failures = []

        with open(input_file_path) as input_file:
            text = input_file.read()
            preprocessed_text = self.preprocess(text)

            # autojunk is too optimal
            sm = SequenceMatcher(None, text, preprocessed_text, autojunk=False)
            
            for tag, i1, i2, j1, j2 in sm.get_opcodes():
                print ("%7s a[%d:%d] (%s) b[%d:%d] (%s)" %
                    (tag, i1, i2, 'a', j1, j2, 'a'))
                if tag == 'delete' or tag == 'equal' or tag == 'replace':
                    print('\n', text[i1:i2], '\n-----------------\n', preprocessed_text[j1:j2], '\n')

            with open(output_file_path, 'w') as output_file:
                output_file.write(preprocessed_text)
            with open(input_ann_file_path, 'r', encoding='utf8') as input_ann_file:
                input_ann_file_content = input_ann_file.read()
                new_content = []
                _start = 0
                _end = 1
                _in_number = False
                ops = sm.get_opcodes()
                for i, ann_char  in enumerate(input_ann_file_content):
                    if not _in_number:
                        if ann_char.isdigit():
                            _start = i
                            _in_number = True
                        else:
                            new_content.append(ann_char)
                    else:
                        if not ann_char.isdigit():
                            _end = i
                            _in_number = False
                            if not input_ann_file_content[_start - 1].isalpha():
                                num = int(input_ann_file_content[_start:_end])
                                old_num = num
                                actual_num = len(text.encode('utf-16le')[:(num * 2)].decode('utf-16le'))
                                # print(num)
                                new_num = 'NONE'
                                for tag, i1, i2, j1, j2 in sm.get_opcodes():
                                    if actual_num >= i1 and actual_num <= i2:
                                        if tag == 'equal':
                                            new_num = num - (i1 - j1)
                                            break
                                    if actual_num >= i1 and actual_num < i2:
                                        if tag == 'delete':
                                            new_num = j1
                                            print('on delete:', actual_num, num, new_num)
                                            break
                                if new_num == 'NONE':
                                    failures.append(input_file_path)
                                new_content.append(str(new_num))
                                new_content.append(ann_char)
                            _start = _end + 1
                            continue

                with open(output_ann_file_path, 'w', encoding='utf8') as cleaned_ann:
                    cleaned_ann.write(''.join(new_content))
        if len(failures) > 0:
            print(input_file_path, 'number of failures:', len(failures))
                
    def preprocess(self, text):
        preprocessed_text = text
        if self.po.get('remove_links'):
            preprocessed_text = self.remove_links(preprocessed_text)
        if self.po.get('remove_mentions'):
            preprocessed_text = self.remove_mentions(preprocessed_text)

        if self.po.get('hashtag') or self.po.get('char_normalization'):
            preprocessed_text = self.preprocess_with_processor(preprocessed_text)

        preprocessed_text = self.put_period(preprocessed_text)
        return preprocessed_text

    def remove_mentions(self, input_file, keep_inner=True):
        new_lines = []
        for line in input_file.split('\n'):
            words = line.split()
            new_line = ''
            for i, word in enumerate(words):
                if not self.is_handle(word):
                    if keep_inner:
                        new_line = " ".join(words[i:])
                        break

            new_lines.append(new_line + '\n')
        return ''.join(new_lines)

    def remove_links(self, input_file):
        new_lines = []
        for line in input_file.split('\n'):
            words = line.split()
            new_line = []
            for word in words:
                if not self.is_url(word):
                    new_line.append(word)
            if len(new_line) > 0:
                new_lines.append(' '.join(new_line) + '\n')
        return ''.join(new_lines)

    def replace_hashtags(self, input_file):
        new_lines = []
        for line in input_file.split('\n'):
            words = line.split()
            new_line = []
            for word in words:
                ############
                if word.startswith('#'):
                    hashed_word = hash_fix(word)
                    new_line.append(hashed_word)
                else:
                    new_line.append(word)
                ############
            if len(new_line) > 0:
                new_lines.append(' '.join(new_line) + '\n')
        return ''.join(new_lines)
    
    def put_period(self, input_file):
        new_lines = []
        for line in input_file.split('\n'):
            if len(line) == 0:
                continue
            elif not line[-1] in '?!.':
                new_lines.append(line + '.')
            else:
                new_lines.append(line)
        return '\n'.join(new_lines)
        

    def preprocess_with_processor(self, text):
        lines = text.split('\n')
        new_lines = []
        for line in lines:  
            preprocessed_text = self.text_preprocessor.pre_process_doc(line)
            remove_words = ['<hashtag>','</hashtag>','<repeated>', '<elongated>', '<allcaps>' ,'</allcaps>']
            filtered_words = list(filter(lambda w: w not in remove_words, preprocessed_text))
            new_line = ' '.join(filtered_words)
            new_lines.append(new_line)
        return '\n'.join(new_lines)

    def hash_fix(self, hashtag):
        new_hash = re.sub(r'#', '', hashtag)
        tokens = segment(str(new_hash))
        hashed = ' '.join(map(str, tokens)) 
        return hashed

    def is_handle(self, word):
        return word[0] == '@'

    def is_url(self, word):
        return re.match(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', word) 

In [18]:
def preprocess_and_sync(input_file_path, output_file_path, input_ann_file_path, output_ann_file_path):
    p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.EMOJI)
    
    failures = []
    
    with open(input_file_path) as input_file:
        text = input_file.read()
        preprocessed_text = preprocess(text, {
            'remove_links': True,
            'remove_mentions': True,
            'hashtag': {
                'replacement': True
            },
            'char_normalization': True
        })
        
        # autojunk is too optimal
        sm = SequenceMatcher(None, text, preprocessed_text, autojunk=False)
        
        with open(output_file_path, 'w') as output_file:
            output_file.write(preprocessed_text)
        with open(input_ann_file_path, 'r', encoding='utf8') as input_ann_file:
            input_ann_file_content = input_ann_file.read()
            new_content = []
            _start = 0
            _end = 1
            _in_number = False
            ops = sm.get_opcodes()
            for i, ann_char  in enumerate(input_ann_file_content):
                if not _in_number:
                    if ann_char.isdigit():
                        _start = i
                        _in_number = True
                    else:
                        new_content.append(ann_char)
                else:
                    if not ann_char.isdigit():
                        _end = i
                        _in_number = False
                        if not input_ann_file_content[_start - 1].isalpha():
                            num = int(input_ann_file_content[_start:_end])
                            old_num = num
                            actual_num = len(text.encode('utf-16le')[:(num * 2)].decode('utf-16le'))
                            # print(num)
                            new_num = 'NONE'
                            for tag, i1, i2, j1, j2 in sm.get_opcodes():
                                if actual_num >= i1 and actual_num <= i2:
                                    if tag == 'equal':
                                        new_num = num - (i1 - j1)
                                        break
                                if actual_num >= i1 and actual_num < i2:
                                    if tag == 'delete':
                                        new_num = j1
                                        print('on delete:', num, new_num)
                                        break
                            if new_num == 'NONE':
                                failures.append(input_file_path)
                            new_content.append(str(new_num))
                            new_content.append(ann_char)
                        _start = _end + 1
                        continue
            
            with open(output_ann_file_path, 'w', encoding='utf8') as cleaned_ann:
                cleaned_ann.write(''.join(new_content))
    if len(failures) > 0:
        print(input_file_path, 'number of failures:', len(failures))
                
def preprocess(text, preprocessing_options):
    preprocessed_text = text
    if preprocessing_options.get('remove_links'):
        preprocessed_text = remove_links(preprocessed_text)
    if preprocessing_options.get('remove_mentions'):
        preprocessed_text = remove_mentions(preprocessed_text)
        
    if preprocessing_options.get('hashtag') or preprocessing_options.get('char_normalization'):
        annotate = set()

        if preprocessing_options.get('hashtag'):
            if preprocessing_options.get('hashtag').get('replace'):
                annotate.add('hashtag')
        if preprocessing_options.get('char_normalization'):
            annotate.add('elongated')
            annotate.add('repeated')
    
        annotate.add('censored')

        text_preprocessor = TextPreProcessor(annotate=annotate,
            # corpus from which the word statistics are going to be used for word segmentation 
            segmenter="twitter", 

            # corpus from which the word statistics are going to be used for spell correction
            corrector="twitter", 
            unpack_hashtags=True,  # perform word segmentation on hashtags
            unpack_contractions=True,  # Unpack contractions (can't -> can not)
            spell_correct_elong=False,  # spell correction for elongated words

            # select a tokenizer. You can use SocialTokenizer, or pass your own tokenizer, should take as input a string and return a list of tokens
            tokenizer=SocialTokenizer(lowercase=True).tokenize,

            # list of dictionaries, for replacing tokens extracted from the text,with other expressions. You can pass more than one dictionaries.
            dicts=[emoticons]
        )
        

        
    return preprocessed_text

def remove_mentions(input_file, keep_inner=True):
    new_lines = []
    for line in input_file.split('\n'):
        words = line.split()
        new_line = ''
        for i, word in enumerate(words):
            if not is_handle(word):
                if keep_inner:
                    new_line = " ".join(words[i:])
                    break
                
        new_lines.append(new_line + '\n')
    return ''.join(new_lines)

def remove_links(input_file):
    new_lines = []
    for line in input_file.split('\n'):
        words = line.split()
        new_line = []
        for word in words:
            if not is_url(word):
                new_line.append(word)
        if len(new_line) > 0:
            new_lines.append(' '.join(new_line) + '\n')
    return ''.join(new_lines)

def replace_hashtags(input_file):
    new_lines = []
    for line in input_file.split('\n'):
        words = line.split()
        new_line = []
        for word in words:
            ############
            if word.startswith('#'):
                hashed_word = hash_fix(word)
                new_line.append(hashed_word)
            else:
                new_line.append(word)
            ############
        if len(new_line) > 0:
            new_lines.append(' '.join(new_line) + '\n')
    return ''.join(new_lines)

def preprocess_with_processor(text, text_processor):
    preprocessed_text = text_preprocessor.pre_process_doc(text)
    remove_words = ['<hashtag>','</hashtag>','<repeated>', '<elongated>', '<allcaps>' ,'</allcaps>']
    filtered_words = list(filter(lambda w: w not in remove_words, preprocessed_text))
    return ' '.join(filtered_words)

def hash_fix(hashtag):
    new_hash = re.sub(r'#', '', hashtag)
    tokens = segment(str(new_hash))
    hashed = ' '.join(map(str, tokens)) 
    return hashed

def is_handle(word):
    return word[0] == '@'

def is_url(word):
    return re.match(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', word) 

In [3]:
custom_preprocessor = Preprocessor({
    'remove_links': True,
    'remove_mentions': True,
    'hashtag': {
        'replacement': True
    },
    'char_normalization': True
})

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


/home/burak/anaconda3/envs/im/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [4]:
BASE_FOLDER = 'data/tw_data/PDTB_Annotations_20200205__'
NEW_FOLDER = 'preprocessed_with_points'

In [25]:

files = os.listdir('data/tw_data/PDTB_Annotations_20200205__/cleaned/ann')
for file_name in tqdm(files):
    print(file_name)
    try:
        custom_preprocessor.preprocess_and_sync(
            BASE_FOLDER + '/cleaned/threads/' + file_name,
            BASE_FOLDER + '/' + NEW_FOLDER + '/threads/' + file_name,
            BASE_FOLDER + '/cleaned/ann/' + file_name,
            BASE_FOLDER + '/' + NEW_FOLDER + '/ann/' + file_name
        )
    except Exception as err:
        print(file_name, err)

  0%|          | 0/184 [00:00<?, ?it/s]

028_948124816611139589.branch318.txt.username_text_tabseparated


  1%|          | 2/184 [00:01<02:07,  1.43it/s]

136_950356330128314368.branch21.txt.username_text_tabseparated
030_948126677648932869.branch4.txt.username_text_tabseparated


KeyboardInterrupt: 

In [5]:
BASE_FOLDER = 'data/tw_data/PDTB_Annotations_20200205__'
FILE_NAME = '163_950370379897491456.branch1.txt.username_text_tabseparated'
custom_preprocessor.preprocess_and_sync(
    BASE_FOLDER + '/cleaned/threads/' + FILE_NAME,
    BASE_FOLDER + '/' + NEW_FOLDER + '/threads/' + FILE_NAME,
    BASE_FOLDER + '/cleaned/ann/' + FILE_NAME,
    BASE_FOLDER + '/' + NEW_FOLDER + '/ann/' + FILE_NAME
)


  equal a[0:57] (a) b[0:57] (a)

 Any F/A’s want to form a team for Birmingham @IGFesports  
-----------------
 Any F/A’s want to form a team for Birmingham @IGFesports  

 delete a[57:59] (a) b[57:57] (a)

    
-----------------
  

  equal a[59:70] (a) b[57:68] (a)

 Dm me if so 
-----------------
 Dm me if so 

 delete a[70:71] (a) b[68:68] (a)

   
-----------------
  

  equal a[71:92] (a) b[68:89] (a)

  Retweets Appreciated 
-----------------
  Retweets Appreciated 

 insert a[92:92] (a) b[89:90] (a)
  equal a[92:93] (a) b[90:91] (a)

 
 
-----------------
 
 

 delete a[93:138] (a) b[91:91] (a)

 @Axis_Fy @IGFesports @ShoutsAlot
@SCreatiive  
-----------------
  

  equal a[138:169] (a) b[91:122] (a)

 I don’t team with ppl with egos 
-----------------
 I don’t team with ppl with egos 

 insert a[169:169] (a) b[122:123] (a)
  equal a[169:170] (a) b[123:124] (a)

 
 
-----------------
 
 

 delete a[170:191] (a) b[124:124] (a)

 @Axis_Fy @SCreatiive  
-----------------
  

  equ

In [8]:
sm = SequenceMatcher(None, a, b)

for tag, i1, i2, j1, j2 in s.get_opcodes():
    print ("%7s a[%d:%d] (%s) b[%d:%d] (%s)" %
        (tag, i1, i2, a[i1:i2], j1, j2, b[j1:j2]))

  equal a[0:5] (burak) b[0:5] (burak)
 insert a[5:5] () b[5:11] ( fakat)
  equal a[5:12] ( ozmen ) b[11:18] ( ozmen )
replace a[12:14] (bu) b[18:19] (o)
  equal a[14:21] (radaydi) b[19:26] (radaydi)


In [12]:
dir(tp)

['OPT',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'api',
 'clean',
 'defines',
 'enum',
 'parse',
 'preprocess',
 'set_options',
 'tokenize',
 'utils']